📘 Section 1: Importing Required Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score, classification_report,
    confusion_matrix, ConfusionMatrixDisplay
)
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import GradientBoostingClassifier

📘 Section 2: Loading and Restructuring the Dataset

In [ ]:
# Load raw CSV data (no header)
csv_file = "Dados-SLA.csv"
raw_data = pd.read_csv(csv_file, header=None)

# Define number of clients and expected columns (1 ID + 11 responses)
num_clients = 119
num_factors = 12

# Reshape flat data into structured 2D array
reshaped_data = raw_data.values.reshape(num_clients, num_factors)

# Define column names
columns = [
    "Client ID", "Service Rate", "Completed Orders", "Speed", "Consistency",
    "Flexibility", "Failure Recovery", "Information", "Correct Invoices",
    "Conforming Products", "Correct Quantity", "Overall Satisfaction"
]

# Create a DataFrame with the structured data
data = pd.DataFrame(reshaped_data, columns=columns)

📘 Section 3: Creating the Target Variable

In [ ]:
# Define binary target: 1 for satisfied clients (rating >= 4), 0 otherwise
data["Satisfied"] = (data["Overall Satisfaction"].astype(int) >= 4).astype(int)

# Define independent variables (features) and dependent variable (target)
X = data.drop(columns=["Client ID", "Overall Satisfaction", "Satisfied"])
y = data["Satisfied"]

📘 Section 4: Preprocessing - Data Normalization

In [ ]:
# Display the distribution of satisfaction labels
print("Original label distribution:", np.bincount(y))

# Normalize feature values to range [0, 1]
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

📘 Section 5: Training the Gradient Boosting Classifier

In [ ]:
# Initialize Gradient Boosting model with specified hyperparameters
gb_model = GradientBoostingClassifier(
    n_estimators=300,
    learning_rate=0.02,
    max_depth=4,
    subsample=0.7,
    random_state=42
)

# Fit the model to the entire dataset
gb_model.fit(X_scaled, y)

# Predict using the trained model
y_pred = gb_model.predict(X_scaled)

# Show predicted label distribution
print("Predicted label distribution:", np.bincount(y_pred))

# Evaluate the model performance
print("Accuracy (Gradient Boosting):", accuracy_score(y, y_pred))
print("\nClassification Report:\n", classification_report(y, y_pred))

📘 Section 6: Error Analysis - Misclassified Clients

In [ ]:
# Add prediction and correctness columns to the DataFrame
data["Predicted"] = y_pred
data["Correct"] = (data["Satisfied"] == data["Predicted"]).astype(int)

# Filter and display misclassified clients
misclassified = data[data["Correct"] == 0][["Client ID", "Satisfied", "Predicted"]]
print("\nClients that reduced the model's accuracy:")
print(misclassified)

📘 Section 7: Feature Importance Table

In [ ]:
# Get feature importances from the trained model
importances = gb_model.feature_importances_
importance_df = pd.DataFrame({
    "Factor": X.columns,
    "Importance": importances
}).sort_values(by="Importance", ascending=False).round(3)

# Display the importance table using matplotlib
fig, ax = plt.subplots(figsize=(8, 2.5))
ax.axis("tight")
ax.axis("off")

# Light grey table style
table = ax.table(
    cellText=importance_df.values,
    colLabels=importance_df.columns,
    cellLoc='center',
    loc='center',
    colColours=["#D3D3D3"] * 2
)

# Customize table appearance
table.auto_set_font_size(False)
table.set_fontsize(10)

plt.title("Feature Importance - Gradient Boosting")
plt.show()